# Loading

In [2]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
import openai
import string
import json
import copy
import re
import time

import os
from dotenv import load_dotenv


In [22]:
load_dotenv()
model="gemini-1.5-flash"

gemini_api_key = os.getenv("GEMINI_API_KEY_1_5_SIX")

client = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

try:
    resp = client.chat.completions.create(
        model="gemini-1.5-flash",
        messages=[{"role": "user", "content": "Ping"}]
    )
    print("Success:", resp.choices[0].message.content)
except Exception as e:
    # <-- note the closing ')' here
    print("Request failed:", type(e).__name__, e)

Success: Pong!



# Seniority

In [23]:
train_df = pd.read_csv('../data/seniority_labelled_development_set_cleaned.csv')
test_df = pd.read_csv('../data/seniority_labelled_test_set_cleaned.csv')

In [7]:
train_df.iloc[1131]

job_id                                                             33027740
job_title                                                   Fitter & Welder
job_summary               A food manufacturing company has an opportunit...
job_ad_details            A well known food manufacturer operating since...
classification_name                    Manufacturing, Transport & Logistics
subclassification_name                                                Other
y_true                                                          experienced
Name: 1131, dtype: object

# RBIC Functions

### RB


In [8]:
#== step 1: role explanation
def step_1():
    messages = [
        {"role": "system", "content": "You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, intermediate to senior, senior executive] . If not present in the set, then create a label."},
        {"role": "user", "content": "Based on your role, can you briefly explain what seniority means, and what seniority labels look like?"}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})

    return response.choices[0].message.content, messages

In [9]:
#== few-shot examples
def fewshot(messages):
    # messages.append({
    #     "role": "user",
    #     "content": "I will provide you with some examples on how to accomplish your task"
    # })

    # response = client.chat.completions.create(
    #     model=model,
    #     messages=messages
    # )
    # messages.append({"role": "assistant", "content": response.choices[0].message.content})

    few_shot_indices = [2081,1,5,12,3]

    for i in few_shot_indices:
        desc = {
          "job_title": train_df.iloc[i].job_title,
          "job_summary": train_df.iloc[i].job_summary,
          "job_ad_details": train_df.iloc[i].job_ad_details,
          "classification_name": train_df.iloc[i].classification_name,
          "subclassification_name": train_df.iloc[i].subclassification_name
        }
        desc_str = str(desc)

        # add the description
        messages.append({
          "role": "user",
          "content": desc_str
        })

        label = train_df.iloc[i].y_true
        label_str = str(label)

        # add the output
        messages.append({
            "role": "assistant",
            "content": label_str
        })

    return messages

In [10]:
#== step 2: setting sub-task --> ask for seniority patterns
def step_2(messages):
    messages.append({
        "role": "user",
        "content": "As a seniority label predictor, what are some common phrases or patterns that indicate a seniority label in a job description?"
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})

    return response.choices[0].message.content, messages

### IC

In [11]:
#== step 3: presence of seniority (skipped)
def step_3(messages_static, desc_str):
    messages = copy.deepcopy(messages_static)

    messages.append({
        "role": "user",
        "content": f"{desc_str} does this job description include any seniority-related information? Just respond with 'Yes' or 'No'."
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})

    p3_content = response.choices[0].message.content

    # clean and check the response
    response_p3 = p3_content.translate(str.maketrans('', '', string.punctuation))
    response_p3 = response_p3.strip().lower()[:3]
    #print(f"step3 messages: {len(messages)}")
    return response_p3, messages

In [12]:
#== step 4: iterative coaching/finding clues to prevent hallucination
def step_4(response_p3, messages):
    messages.append({
            "role": "user",
            "content": "Extract the seniority-related phrases from the text verbatim. Respond in JSON: {\"Clue\": \"\"}."
        })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})
    #print(f"step4 messages: {len(messages)}")

    return response.choices[0].message.content, messages

In [13]:
#== step 5: use the clue to generate the final output
def step_5(messages):
    messages.append({
        "role": "user",
        "content": (
            "Based on the extracted seniority clue, return a structured seniority label in the format {\"seniority_label\": \"seniority label\"}. "
                        
        )
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    answer_str = response.choices[0].message.content

    print(messages)

    return answer_str

In [14]:
def RBIC_static_messages(verbose=False, add_fewshot=True):
    response, messages = step_1()
    if verbose: print(f"RB step 1: {response}\n")

    response, messages = step_2(messages)
    if verbose: print(f"RB step 2: {response}\n")

    if add_fewshot:
        messages = fewshot(messages)
        if verbose: print(f"Fewshot examples added\n")
    return messages

In [15]:
def RBIC(messages, desc_str, verbose=False):
    response_p3, messages_local = step_3(messages, desc_str)
    if verbose: print(f"IC step 1: {response_p3}\n")

    response, messages_local = step_4(response_p3, messages_local)
    if verbose: print(f"IC step 2: {response}\n")

    answer_str = step_5(messages_local)

    

    match = re.search(r'\{.*\}', answer_str)
    
    json_str = match.group(0)



    data = json.loads(json_str)
    

    # extract the value
    label = data["seniority_label"]
    
    if verbose: print(f"IC step 3 (Final): {label}\n")
    if verbose: print(f"IC step 3 (Final Raw): {answer_str}\n")

    return str(label)

# Testing

### Qualitative Tests

In [16]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=True)

RB step 1: Seniority, in the context of job descriptions, refers to the level of experience and responsibility expected of a candidate.  It indicates the career stage a position falls into – from entry-level to executive positions.

Seniority labels can be explicit (e.g., "Senior Software Engineer," "Entry-Level Accountant") or implicit, requiring inference based on the job description's requirements, responsibilities, and required skills.  Explicit labels often use words like:

* **Entry-level/Junior:**  Little to no prior experience needed.  Focus is on training and foundational skills.
* **Intermediate/Mid-level:** Requires some experience and demonstrated proficiency in core skills.  Often involves increased responsibility compared to entry-level.
* **Senior/Lead/Principal/Head:**  Requires significant experience, advanced skills, mentorship capabilities, and often project/team leadership.
* **Executive/Director/VP:**  Top-level positions involving strategic planning, oversight of 

In [23]:
ind = 395

#[2081,1,5,12,3]

desc = {
    "job_title": test_df.iloc[ind].job_title,
    "job_summary": test_df.iloc[ind].job_summary,
    "job_ad_details": test_df.iloc[ind].job_ad_details,
    "classification_name": test_df.iloc[ind].classification_name,
    "subclassification_name": test_df.iloc[ind].subclassification_name,
  }
desc_str = str(desc)

label = test_df.iloc[ind].y_true
label_str = str(label)

print(f"len of messages_static {len(messages_static)}")
label_pred = RBIC(messages_static, desc_str, verbose=True)


print(f"pred = {label_pred}")
print(f"truth = {label_str}")

len of messages_static 15
IC step 1: yes

IC step 2: ```json
{"Clue": ["solution leadership role", "ensure a robust and fit for purpose IM solution architecture", "work closely with the business and technology stakeholders", "ensure best practice and robust solution design and governance for the IM platform", "hands-on experience implementing Big data solutions and technologies", "engage senior stakeholders"]}
```


[{'role': 'system', 'content': 'You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/

In [24]:
test_df.iloc[ind].job_ad_details

'Company description:We’re an energy delivery service that owns and operates Victoria’s largest network of electricity and gas infrastructure. We move energy through these networks from where it’s made to where it’s used in more than one million homes and businesses. What we do matters to our nation. At AusNet Services, you’re part of a diverse, inclusive and collaborative team that’s dedicated to bringing safe, efficient and reliable energy to millions of Australians, and you can be certain that safety comes first in everything we do.Job description:What you will be delivering: This great opportunity is solution leadership role within the Enterprise Information Management (IM) CoE and has responsibility for ensuring a robust and fit for purpose IM solution architecture that meets current and future needs of our business.You will work closely with the business and technology stakeholders, ensure best practice and robust solution design and governance for the IM platform. Your hands-on 

### Quantitative Tests

In [25]:
print(test_df.iloc[395])

job_id                                                             38783395
job_title                                     Azure Big Data Technical Lead
job_summary               A solution leadership role with the responsibi...
job_ad_details            Company description:We’re an energy delivery s...
classification_name                  Information & Communication Technology
subclassification_name                                           Architects
y_true                                                                 lead
Name: 395, dtype: object


In [26]:
# df to store model predictions
test_pred_df = pd.DataFrame(columns=["y_pred"])
test_pred_df



,y_pred


In [25]:
test_pred_df2 = pd.DataFrame(columns=["y_pred"])
test_pred_df2


,y_pred


In [27]:
for i in tqdm(range(200)):

    desc = {
        "job_title":             test_df.iloc[i].job_title,
        "job_summary":           test_df.iloc[i].job_summary,
        "job_ad_details":        test_df.iloc[i].job_ad_details,
        "classification_name":    test_df.iloc[i].classification_name,
        "subclassification_name": test_df.iloc[i].subclassification_name
    }
    desc_str = str(desc)

    # retry this same row until it succeeds
    while True:
        try:
            label_pred = RBIC(messages_static, desc_str)
            break
        except Exception as e:
            print(f"Error on row {i} ({e.__class__.__name__}): {e}. "
                  "Sleeping 60 s before retry…")
            time.sleep(60)

    # append the successful prediction as a new row
    test_pred_df2.loc[len(test_pred_df2)] = label_pred




  0%|          | 1/200 [00:03<10:11,  3.07s/it]

[{'role': 'system', 'content': 'You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, 

  1%|          | 2/200 [00:07<12:33,  3.81s/it]

[{'role': 'system', 'content': 'You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, 

  2%|▏         | 3/200 [01:12<1:44:55, 31.95s/it]

...

[{'role': 'system', 'content': 'You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, 

100%|█████████▉| 199/200 [45:46<00:09,  9.76s/it]

[{'role': 'system', 'content': 'You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, 

100%|██████████| 200/200 [45:48<00:00, 13.74s/it]

[{'role': 'system', 'content': 'You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, 

In [29]:
test_pred_df2

,y_pred
0,senior
1,senior
2,entry-level
3,senior
4,entry-level
...,...
195,experienced
196,senior
197,intermediate
198,intermediate


In [30]:
test_pred_df2.to_csv('seniority_labelled_test_set_gemini_preds_few_shot_part1.csv', index=False)

# Testing 

We perform Exact Matches and categorise the non exact matches to categories to ensure a match of similar labels. 

In [31]:
import pandas as pd


df_true = pd.read_csv("seniority_labelled_test_set_ans.csv")     
df_pred = pd.read_csv("seniority_labelled_test_set_gemini_preds_few_shot.csv")    



df = pd.concat([df_true.reset_index(drop=True), 
                df_pred.reset_index(drop=True)], 
               axis=1)


matches   = (df["y_true"] == df["y_pred"]).sum()
total     = len(df)
mismatches = total - matches

print(f"Exact matches:   {matches} / {total}")
print(f"Non-matches:     {mismatches} / {total}")


mismatch_df = df[df["y_true"] != df["y_pred"]]


for idx, row in mismatch_df.iterrows():
    print(f"Row {idx}: true={row['y_true']}  pred={row['y_pred']}")

Exact matches:   361 / 689
Non-matches:     328 / 689
Row 1: true=experienced  pred=senior
Row 4: true=intermediate  pred=entry-level
Row 6: true=experienced  pred=intermediate
Row 11: true=experienced  pred=intermediate
Row 15: true=experienced  pred=senior
Row 16: true=experienced  pred=intermediate
Row 17: true=intermediate  pred=entry-level
Row 22: true=experienced  pred=senior
Row 30: true=head  pred=senior
Row 31: true=senior  pred=supervisor
Row 34: true=experienced  pred=standard
Row 37: true=trainee  pred=intermediate
Row 39: true=intermediate  pred=mid-level
Row 40: true=experienced  pred=intermediate
Row 42: true=experienced  pred=intermediate
Row 46: true=experienced  pred=intermediate
Row 47: true=assistant  pred=junior
Row 49: true=intermediate  pred=entry-level
Row 50: true=experienced assistant  pred=intermediate
Row 51: true=assistant  pred=entry-level
Row 52: true=intermediate  pred=entry-level to intermediate
Row 53: true=lead  pred=mid-level
Row 54: true=entry-level

Saving non exact matches for further test

In [ ]:
# Save only the rows where y_true != y_pred
mismatch_df.to_csv("seniority_mismatches_gemini.csv", columns=["y_true", "y_pred"], index=False)

Non exact matches categorised. 

In [13]:
import pandas as pd
df = pd.read_csv("seniority_mismatches_gemini.csv")

# Define category mapping
def map_sen(label):
    lab = str(label).lower()
    if 'entry' in lab:
        return 'Entry'
    elif 'junior' in lab or 'assistant' in lab:
        return 'Junior'
    elif 'intermediate' in lab or 'experienced' in lab or 'mid' in lab:
        return 'Mid'
    elif 'senior' in lab or 'lead' in lab:
        return 'Senior'
    elif any(x in lab for x in ['manager','director','chief','head','executive','principal']):
        return 'Leadership'
    else:
        return 'Other'

# Apply mapping
df['true_cat'] = df['y_true'].apply(map_sen)
df['pred_cat'] = df['y_pred'].apply(map_sen)
df['cat_match'] = df['true_cat'] == df['pred_cat']

# Calculate overall match percentage
overall_match_pct = df['cat_match'].mean() * 100

print("matches in non exact matches:",overall_match_pct)

# Breakdown by true category match percentage
breakdown = df.groupby('true_cat')['cat_match'].mean().reset_index()
breakdown['match_pct'] = breakdown['cat_match'] * 100
breakdown = breakdown[['true_cat', 'match_pct']]



matches in non exact matches: 41.15853658536585


# Regular prompting

In [60]:
# df to store model predictions
test_pred_df8 = pd.DataFrame(columns=["y_pred"])
test_pred_df8

,y_pred


In [ ]:
messages_static = [
    {"role": "system", "content": "You are an expert job ad annotator. Your task is to extract seniority information from job descriptions."},
]

In [61]:
import copy
import json


for i in tqdm(range(200)):
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_summary": test_df.iloc[i].job_summary,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "classification_name": test_df.iloc[i].classification_name,
        "subclassification_name": test_df.iloc[i].subclassification_name,
    }
    desc_str = str(desc)

    # rebuild messages for this call
    messages = copy.deepcopy(messages_static)
    messages.append({
        "role": "user",
        "content": (
            f"{desc_str} infer the seniority label from this job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, intermediate to senior, senior executive] . If not present in the set, then create a label. "
            "Return a structured seniority label in the format {\"seniority_label\": \"seniority label\"}."
        )
    })

    # retry on any error, pausing 60s before each retry
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages
            )
            break
        except Exception as e:
            print(f"Error on row {i} ({e.__class__.__name__}): {e}. Sleeping 60s before retry…")
            time.sleep(60)

    answer_str = response.choices[0].message.content

    # parse out the JSON block
    try:
            match = re.search(r'\{.*\}', answer_str)
    
            json_str = match.group(0)

            data = json.loads(json_str)
    

            # extract the value
            label = data["seniority_label"]
    
    except (ValueError, json.JSONDecodeError, KeyError) as parse_err:
        print(f"Failed to parse JSON on row {i}: {answer_str}")
        label = "ERROR " + answer_str

    # append to next empty row
    test_pred_df8.loc[len(test_pred_df8)] = label



  8%|▊         | 16/200 [00:11<01:58,  1.55it/s]

Error on row 16 (RateLimitError): Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '21s'}]}}]. Sleeping 60s before retry…


 16%|█▌        | 31/200 [01:24<02:10,  1.29it/s]

Error on row 31 (RateLimitError): Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '9s'}]}}]. Sleeping 60s before retry…


 24%|██▎       | 47/200 [02:37<01:46,  1.43it/s]

...

 94%|█████████▍| 188/200 [17:56<02:55, 14.59s/it]

Failed to parse JSON on row 187: ```json
{"salary": "100000-0-AUD-YEAR"}
```

```json
{"seniority_label": "manager"}
```



 98%|█████████▊| 197/200 [18:03<00:03,  1.23s/it]

Error on row 197 (RateLimitError): Error code: 429 - [{'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}]. Sleeping 60s before retry…


100%|██████████| 200/200 [19:07<00:00,  5.74s/it]


In [62]:
test_pred_df8

,y_pred
0,senior
1,senior
2,assistant
3,senior
4,entry-level
...,...
195,experienced
196,senior
197,associate
198,mid-level


In [63]:
# export the dataframe
test_pred_df8.to_csv(
    'seniority_labelled_test_set_gemini_preds_simple_prompt_part1.csv',
    index=False
)


In [66]:
import pandas as pd


df_true = pd.read_csv("seniority_labelled_test_set_ans.csv")     
df_pred = pd.read_csv("seniority_labelled_test_set_gemini_preds_simple_prompt.csv")    



df = pd.concat([df_true.reset_index(drop=True), 
                df_pred.reset_index(drop=True)], 
               axis=1)


matches   = (df["y_true"] == df["y_pred"]).sum()
total     = len(df)
mismatches = total - matches

print(f"Exact matches:   {matches} / {total}")
print(f"Non-matches:     {mismatches} / {total}")


mismatch_df = df[df["y_true"] != df["y_pred"]]


for idx, row in mismatch_df.iterrows():
    print(f"Row {idx}: true={row['y_true']}  pred={row['y_pred']}")

Exact matches:   404 / 689
Non-matches:     285 / 689
Row 1: true=experienced  pred=senior
Row 2: true=entry-level  pred=assistant
Row 4: true=intermediate  pred=entry-level
Row 17: true=intermediate  pred=entry-level
Row 19: true=entry-level  pred=experienced
Row 30: true=head  pred=manager
Row 31: true=senior  pred=experienced
Row 35: true=entry-level  pred=independent
Row 38: true=entry-level  pred=experienced
Row 39: true=intermediate  pred=coordinator
Row 41: true=entry-level  pred=assistant
Row 43: true=lead  pred=senior
Row 48: true=intermediate  pred=senior
Row 50: true=experienced assistant  pred=experienced
Row 51: true=assistant  pred=entry-level
Row 52: true=intermediate  pred=experienced
Row 53: true=lead  pred=intermediate
Row 54: true=entry-level  pred=intermediate
Row 56: true=senior  pred=manager
Row 59: true=assistant  pred=assistant manager
Row 60: true=intermediate  pred=mid-level
Row 61: true=entry-level  pred=independent
Row 62: true=senior  pred=manager
Row 66: t

In [67]:
mismatch_df.to_csv("seniority_mismatches_gemini_simple_prompt.csv", columns=["y_true", "y_pred"], index=False)

In [17]:
import pandas as pd 

df = pd.read_csv("seniority_mismatches_gemini_simple_prompt.csv", header=None, names=['y_true','y_pred'])

def map_custom(label):
    lab = str(label).lower()
    if any(x in lab for x in ['entry', 'graduate', 'trainee', 'cadet', 'apprentice']):
        return 'Entry'
    if any(x in lab for x in ['assistant', 'junior', 'student']):
        return 'Early Career'
    if any(x in lab for x in ['intermediate', 'experienced', 'mid']):
        return 'Mid Career'
    if any(x in lab for x in ['senior', 'lead', 'principal', 'specialist', 'sous']):
        return 'Senior'
    if any(x in lab for x in ['manager', 'director', 'executive', 'chief', 'head', 'owner-operator', 'contract', 'coordinator', 'business leader', 'team leader', 'board']):
        return 'Management / Exec'
    return 'Other'

df['true_cat'] = df['y_true'].apply(map_custom)
df['pred_cat'] = df['y_pred'].apply(map_custom)
df['match'] = df['true_cat'] == df['pred_cat']

matches = df['match'].sum()
total = len(df)
non_matches = total - matches
overall_pct = matches / total * 100

print("Out of the mismatches, the following percentage is the matches",overall_pct)

Out of the mismatches, the following percentage is the matches 44.40559440559441
